In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from IPython.display import clear_output, display
import sys
sys.path.insert(0,'..')
from celeba_reader import CelebaSequence

In [ ]:
def face_classifier(images_batch):
    conv1 = tf.layers.conv2d(
        inputs=images_batch,
        filters=8,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=4, strides=4)

    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=16,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=4, strides=4)

    pool2_flat = tf.reshape(pool2, [-1, 2288])
    
    dense1 = tf.layers.dense(inputs=pool2_flat, units=256, activation=tf.nn.relu)    
    dense2 = tf.layers.dense(inputs=dense1, units=256, activation=tf.nn.relu)
    dense3 = tf.layers.dense(inputs=dense2, units=32, activation=tf.nn.relu)
    predictions = tf.layers.dense(inputs=dense3, units=1, activation=tf.nn.sigmoid)
    
    predictions = tf.reshape(predictions, [-1])
    return predictions

In [ ]:
sess = tf.InteractiveSession()

images_batch = tf.placeholder(tf.float32, [None, 218, 178, 3])
labels = tf.placeholder(tf.int32, [None])
classifier = face_classifier(images_batch)

loss = tf.losses.log_loss(labels=labels, predictions=classifier)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

In [ ]:
# saver.restore(sess, './checkpoints/model.ckpt')

In [ ]:
paths = {
    'img_path': '/path/to/img_align_celeba',
    'attrs_path': '../list_attr_celeba.csv',
    'partition_path': '../list_eval_partition.csv'
}
batch = 100
train_data = CelebaSequence(partition=0, batch=batch, **paths)

In [ ]:
def train():
    i = 0
    for x, y in train_data:
        sess.run(train_op, feed_dict={images_batch: x, labels: y})
        if i % 10 == 0:
            clear_output()
            display(i)
        if i % 100 == 0:
            save_path = saver.save(sess, 'checkpoints/model.ckpt')
            print("Model saved in path: %s" % save_path)
        if i == 1000:
            break
        i = i + 1

In [ ]:
train()

In [ ]:
inputs = {
    "images_batch": images_batch,
    "labels": labels
}
outputs = {"predictions": classifier}
tf.saved_model.simple_save(sess, './tmp', inputs, outputs)